In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

#Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt


#Modeling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder



from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

#Evaluation
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


from scipy import stats

#Set figure size and figure size for all plots
plt.rc("figure", figsize = (16,16))
plt.rc("font", size=14)

# Allow all columns to be displayed
pd.set_option('display.max_columns', None)

# Project purpose

The goal of this project is to use features captured from survey data to predict how likely individuals are to receive their H1N1 and seasonal flu vaccines.

Specifically, I will be predicting two probabilities:
- h1n1_vaccine - Whether respondent received H1N1 flu vaccine.
- seasonal_vaccine - Whether respondent received seasonal flu vaccine.
> Both are binary variables: 0 = No; 1 = Yes. Some respondents didn't get either vaccine, others got only one, and some got both. This is formulated as a multilabel (and not multiclass) problem.

# Project Plan

1. Aquisition
  * Download data into local drive
2. Prepare
  * Read in data csv using pandas
  * Chekck data types and null values
  * Fill in nulls
  * Encode appropriately
  * Scale if needed
3. Explore
4. Modeling
5. Conclusions

# Acquire

- I downloaded the data from https://www.drivendata.org/competitions/66/flu-shot-learning/data/ into the same file where this analysis is being conducted
- Now I'll turn both the feature csv and the target variable csv data into a pandas dataframes for analysis and modeling

In [2]:
# Read in the training set feature csv using pandas
df = pd.read_csv('Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Features.csv', index_col=0)

Take a look at the data

In [3]:
df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,4.0,2.0,5.0,4.0,2.0,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN


In [4]:
target_variable_df = pd.read_csv('Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Labels.csv', index_col=0)

In [5]:
target_variable_df

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0
...,...,...
26702,0,0
26703,0,0
26704,0,1


- The target variable csv will need to be concatenated onto the feature csv

In [6]:
df = pd.concat([df, target_variable_df], axis = 1)

In [7]:
# df = df.drop(columns = 'respondent_id')

Verify it was added correctly

In [8]:
df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN,0,0
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,0,0
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,4.0,2.0,5.0,4.0,2.0,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN,0,1


Success!

### The initial datatframe has:
- 26,707 rows where each row is one person
- 35 columns where each column is a feature this person has indicated on their survey


Let's get some more info about the datframe. We'll use .info() to see how many non-null values we have in each column and what the data types are

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 37 columns):
h1n1_concern                   26615 non-null float64
h1n1_knowledge                 26591 non-null float64
behavioral_antiviral_meds      26636 non-null float64
behavioral_avoidance           26499 non-null float64
behavioral_face_mask           26688 non-null float64
behavioral_wash_hands          26665 non-null float64
behavioral_large_gatherings    26620 non-null float64
behavioral_outside_home        26625 non-null float64
behavioral_touch_face          26579 non-null float64
doctor_recc_h1n1               24547 non-null float64
doctor_recc_seasonal           24547 non-null float64
chronic_med_condition          25736 non-null float64
child_under_6_months           25887 non-null float64
health_worker                  25903 non-null float64
health_insurance               14433 non-null float64
opinion_h1n1_vacc_effective    26316 non-null float64
opinion_h1n1_risk          

### Initial thoughts for cleaning/prep
1. Several columns will need to be one hot encoded or label encoded
2. Nans in several columns needs to be handled
3. May need to drop certain columns
4. All the datatypes seem to be appropriate

# Prepare

I'll start by splitting the data into train and test to avoid exploring the test data which is meant to stay unseen. Then I'll perform the same clean and prep changes to each dataframe sequentially.
> There's enough data here to split this further into train and validate sets which will help to prevent overfitting by allowing signs of overfitting to be caught before applying the model to the test data. After the appropriate cleaning and preparation are complete, I will create a validate set.

Now I'll check to see which columns have nans, how many there are, and explore the best ways to fill those nans

In [10]:
#Write a for loop to determine what percentage of each column are nans
def percent_nans(df):
    x = ['column','n_nans', 'percentage_nans']
    missing_data_df = pd.DataFrame(columns=x)
    columns = df.columns
    for col in columns:
        column_name = col
        missing_data = df[col].isnull().sum()
        missing_in_percentage = (df[col].isnull().sum()/df[col].shape[0])*100
        
        missing_data_df.loc[len(missing_data_df)] = [column_name, missing_data, missing_in_percentage]
    return missing_data_df.sort_values(by = 'percentage_nans')

In [11]:
percent_nans(df)

,column,n_nans,percentage_nans
36,seasonal_vaccine,0,0.000000
30,census_msa,0,0.000000
29,hhs_geo_region,0,0.000000
24,sex,0,0.000000
23,race,0,0.000000
21,age_group,0,0.000000
35,h1n1_vaccine,0,0.000000
4,behavioral_face_mask,19,0.071142
5,behavioral_wash_hands,42,0.157262
2,behavioral_antiviral_meds,71,0.265848


- It looks like only three columns have a significant number of values missing:
  - __health_insurance__
  - __employment_industry__
  - __employment_occupation__
- Look at the most common values in each feature and decide if there is a reasonable way to fill the null values

In [12]:
#What is the most common health insurance?
df.health_insurance.value_counts()

1.0    12697
0.0     1736
Name: health_insurance, dtype: int64

In [13]:
#What is the most common employment_industry?
df.employment_industry.value_counts()

fcxhlnwr    2468
wxleyezf    1804
ldnlellj    1231
pxcmvdjn    1037
atmlpfrs     926
arjwrbjb     871
xicduogh     851
mfikgejo     614
vjjrobsf     527
rucpziij     523
xqicxuve     511
saaquncn     338
cfqqtusy     325
nduyfdeo     286
mcubkhph     275
wlfvacwt     215
dotnnunm     201
haxffmxo     148
msuufmds     124
phxvnwax      89
qnlwzans      13
Name: employment_industry, dtype: int64

In [14]:
#What is the most common employment occupation?
df.employment_occupation.value_counts()

xtkaffoo    1778
mxkfnird    1509
emcorrxb    1270
cmhcxjea    1247
xgwztkwe    1082
hfxkjkmi     766
qxajmpny     548
xqwwgdyp     485
kldqjyjy     469
uqqtjvyb     452
tfqavkke     388
ukymxvdu     372
vlluhbov     354
oijqvulv     344
ccgxvspp     341
bxpfxfdn     331
haliazsg     296
rcertsgn     276
xzmlyyjv     248
dlvbwzss     227
hodpvpew     208
dcjcmpih     148
pvmttkik      98
Name: employment_occupation, dtype: int64

It is evident that __employment_industry__ and __employment_occupation__ do not have a single most overwhelming industry or occupation that could be reasonably used to fill in the remaining half empty values. I will drop these columns and not include them in the analysis

On the other hand, __health_insurance__ has 12,697 observation recoded as having insurance of the total 14,433 observations with values recorded. It may be reasonable to simply fill in the remaining 45% missing values with the label for having insurance.

At this point I need to split the data into train and test. I think it would be better for exploring and modeling purposes to have a train and test dataframe for each target variable. After the predictions are made, I will concat the dataframes together

In [15]:
df.seasonal_vaccine.value_counts()

0    14272
1    12435
Name: seasonal_vaccine, dtype: int64

In [16]:
#Create two dataframes each with only one of the target variables
h1n1_df = df.drop(columns = 'seasonal_vaccine')

seasonal_df = df.drop(columns = 'h1n1_vaccine')

In [17]:
# Use the train test split function from Sklearn and add a random seed for reproducibility
# Use Stratify y parameter to ensure the same proportion of the y variable in both train and testt dfs
h1n1_train, h1n1_test = train_test_split(h1n1_df, random_state=123, train_size=.80, stratify=h1n1_df.h1n1_vaccine)

In [18]:
# Use the train test split function from Sklearn and add a random seed for reproducibility
# Stratify y parameter to ensure the same proportion of the y variable in both train and testt dfs
seasonal_train, seasonal_test = train_test_split(seasonal_df, random_state=123, train_size=.80, stratify=seasonal_df.seasonal_vaccine)

### Drop the employment_industry and employment_occupation columns from dataframes

In [19]:
h1n1_train = h1n1_train.drop(columns =['employment_industry', 'employment_occupation'] )
h1n1_test = h1n1_test.drop(columns =['employment_industry', 'employment_occupation'] )

Check to make sure the columns were dropped

In [20]:
h1n1_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,4.0,2.0,2.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0,0
2876,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,5.0,4.0,5.0,5.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.0,0.0,0
22939,2.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,2.0,1.0,0


In [21]:
h1n1_test.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1372,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.0,0.0,0
21526,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,2.0,4.0,4.0,2.0,4.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Principle City",1.0,0.0,0
396,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,3.0,1.0,2.0,3.0,1.0,2.0,35 - 44 Years,College Graduate,White,Male,NaN,NaN,NaN,Employed,lzgpxyit,Non-MSA,1.0,2.0,0


In [22]:
seasonal_train = seasonal_train.drop(columns =['employment_industry', 'employment_occupation'] )
seasonal_test = seasonal_test.drop(columns =['employment_industry', 'employment_occupation'] )

In [23]:
seasonal_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22862,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,45 - 54 Years,< 12 Years,Hispanic,Male,"<= $75,000, Above Poverty",Married,Rent,Not in Labor Force,kbazzjca,Non-MSA,0.0,0.0,0
20471,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,2.0,5.0,4.0,1.0,18 - 34 Years,College Graduate,Other or Multiple,Male,"> $75,000",Not Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",2.0,0.0,0
11401,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,5.0,5.0,5.0,4.0,4.0,4.0,18 - 34 Years,< 12 Years,Hispanic,Female,Below Poverty,Married,Rent,Not in Labor Force,atmpeygn,"MSA, Not Principle City",1.0,2.0,0


In [24]:
seasonal_test.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22420,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,5.0,2.0,1.0,4.0,2.0,2.0,55 - 64 Years,Some College,White,Male,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",3.0,0.0,0
24211,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,5.0,5.0,2.0,2.0,18 - 34 Years,Some College,Hispanic,Male,NaN,Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,2.0,0
3799,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,3.0,2.0,1.0,3.0,1.0,1.0,55 - 64 Years,12 Years,White,Female,NaN,Not Married,Own,Employed,fpwskwrf,"MSA, Principle City",1.0,0.0,0


It looks like the columns were dropped appropriately. Now let's fill in null values in the remaining columns

In [25]:
#Write a function to fill in the null values with the most common occurence
def fill_null_values(train, test):
    train = train.apply(lambda x:x.fillna(x.value_counts().index[0]))
    test = test.apply(lambda x:x.fillna(x.value_counts().index[0]))

    return train, test

In [26]:
h1n1_train, h1n1_test = fill_null_values(h1n1_train, h1n1_test)

Verify that there are no more null values in either dataframe

In [27]:
h1n1_train.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [28]:
h1n1_test.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [29]:
seasonal_train, seasonal_test = fill_null_values(seasonal_train, seasonal_test)

In [30]:
seasonal_train.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [31]:
seasonal_test.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

### Encoding

Next we need to encode the columns that have objects as values and turn them into integer representations for the purpose of the classification model to predict how likely people are to get their flu vaccines

Look at which columns need to be encoded

In [32]:
h1n1_train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,4.0,2.0,2.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0,0
2876,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,4.0,5.0,5.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.0,0.0,0
22939,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,2.0,1.0,0
4154,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,1.0,5.0,1.0,1.0,45 - 54 Years,< 12 Years,Other or Multiple,Male,"> $75,000",Married,Rent,Employed,mlyzmhmf,"MSA, Principle City",1.0,0.0,0
23434,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,2.0,1.0,18 - 34 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",1.0,3.0,0
4285,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,4.0,1.0,45 - 54 Years,Some College,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Unemployed,bhuqouqj,"MSA, Not Principle City",3.0,0.0,0
14877,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,1.0,4.0,2.0,1.0,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",1.0,2.0,0


In [33]:
# def label_encode_columns(train, test):

#     encoder = LabelEncoder()
   
#     encode_list = ['rent_or_own', 'employment_status', 'marital_status', 'sex']
    
             
#     for column in encode_list:
#         train[column] = encoder.fit_transform(train[column])
#         test[column] = encoder.transform(test[column])

#         return train, test

In [34]:
# train, test = label_encode_columns(train, test)

In [35]:
def label_encode_columns(train, test):

    encoder = LabelEncoder()
   
    train['encoded_rent_or_own'] = encoder.fit_transform(train['rent_or_own'])
    train['encoded_marital_status'] = encoder.fit_transform(train['marital_status'])
    train['encoded_sex'] = encoder.fit_transform(train['sex'])

    test['encoded_rent_or_own'] = encoder.fit_transform(test['rent_or_own'])
    test['encoded_marital_status'] = encoder.fit_transform(test['marital_status'])
    test['encoded_sex'] = encoder.fit_transform(test['sex'])
    
    return train, test

In [36]:
h1n1_train, h1n1_test = label_encode_columns(h1n1_train, h1n1_test)

In [37]:
h1n1_train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,4.0,2.0,2.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0,0,0,0,1
2876,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,4.0,5.0,5.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.0,0.0,0,1,1,0
22939,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,2.0,1.0,0,1,0,0
4154,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,1.0,5.0,1.0,1.0,45 - 54 Years,< 12 Years,Other or Multiple,Male,"> $75,000",Married,Rent,Employed,mlyzmhmf,"MSA, Principle City",1.0,0.0,0,1,0,1
23434,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,2.0,1.0,18 - 34 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",1.0,3.0,0,0,0,1
4285,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,4.0,1.0,45 - 54 Years,Some College,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Unemployed,bhuqouqj,"MSA, Not Principle City",3.0,0.0,0,0,0,0
14877,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,1.0,4.0,2.0,1.0,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",1.0,2.0,0,0,0,0


In [38]:
h1n1_test

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1372,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.0,0.0,0,0,1,0
21526,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,4.0,2.0,4.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Principle City",1.0,0.0,0,0,0,0
396,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,3.0,1.0,2.0,35 - 44 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,Non-MSA,1.0,2.0,0,0,0,1
11045,3.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,4.0,4.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,bhuqouqj,"MSA, Principle City",3.0,1.0,0,0,0,1
5072,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,2.0,2.0,2.0,65+ Years,College Graduate,White,Male,Below Poverty,Not Married,Rent,Not in Labor Force,oxchjgsf,"MSA, Principle City",0.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20153,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,4.0,4.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",3.0,0.0,0,0,0,0
8108,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,College Graduate,White,Female,"> $75,000",Not Married,Rent,Employed,dqpwygqj,"MSA, Principle City",1.0,0.0,1,1,1,0
19271,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0,5.0,5.0,5.0,55 - 64 Years,Some College,White,Female,Below Poverty,Married,Own,Employed,mlyzmhmf,Non-MSA,3.0,2.0,1,0,0,0


In [39]:
seasonal_train, seasonal_test = label_encode_columns(seasonal_train, seasonal_test)

In [40]:
seasonal_train.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [41]:
seasonal_test.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [42]:
# categorical_cols = ['rent_or_own', 'employment_status', 'marital_status', 'sex']

# categorical_cols

# train[categorical_cols] = train[categorical_cols].apply(lambda col: encoder.fit_transform(col))

In [43]:
# def encode(train, test):
#     # creating instance of one-hot-encoder
#     enc = OneHotEncoder()
#     # passing bridge-types-cat column (label encoded values of bridge_types)
#     enc_df = pd.DataFrame(enc.fit_transform(train[['age_group', 'education', 'race', 'income_poverty']]).toarray())
#     # merge with main df bridge_df on key values
#     train = train.join(enc_df)

#     # passing bridge-types-cat column (label encoded values of bridge_types)
#     enc_df2 = pd.DataFrame(enc.fit_transform(test[['age_group', 'education', 'race', 'income_poverty']]).toarray())
#     # merge with main df bridge_df on key values
#     test = test.join(enc_df2)
    
#     return train, test

In [44]:
# col_name = ['age_group', 'education', 'race', 'income_poverty']

In [45]:
# train, test = encode(train, test)

Look at the dataframe to ensure all the label encoded columns were added correctly

In [46]:
h1n1_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,4.0,2.0,2.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0,0,0,0,1
2876,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,4.0,5.0,5.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.0,0.0,0,1,1,0
22939,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,2.0,1.0,0,1,0,0


In [47]:
h1n1_test.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1372,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.0,0.0,0,0,1,0
21526,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,4.0,2.0,4.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Principle City",1.0,0.0,0,0,0,0
396,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,3.0,1.0,2.0,35 - 44 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,Non-MSA,1.0,2.0,0,0,0,1


In [48]:
seasonal_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,seasonal_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22862,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,45 - 54 Years,< 12 Years,Hispanic,Male,"<= $75,000, Above Poverty",Married,Rent,Not in Labor Force,kbazzjca,Non-MSA,0.0,0.0,0,1,0,1
20471,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,2.0,5.0,4.0,1.0,18 - 34 Years,College Graduate,Other or Multiple,Male,"> $75,000",Not Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",2.0,0.0,0,0,1,1
11401,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,5.0,5.0,4.0,4.0,4.0,18 - 34 Years,< 12 Years,Hispanic,Female,Below Poverty,Married,Rent,Not in Labor Force,atmpeygn,"MSA, Not Principle City",1.0,2.0,0,1,0,0


In [49]:
seasonal_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,seasonal_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22862,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,45 - 54 Years,< 12 Years,Hispanic,Male,"<= $75,000, Above Poverty",Married,Rent,Not in Labor Force,kbazzjca,Non-MSA,0.0,0.0,0,1,0,1
20471,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,2.0,5.0,4.0,1.0,18 - 34 Years,College Graduate,Other or Multiple,Male,"> $75,000",Not Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",2.0,0.0,0,0,1,1
11401,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,5.0,5.0,4.0,4.0,4.0,18 - 34 Years,< 12 Years,Hispanic,Female,Below Poverty,Married,Rent,Not in Labor Force,atmpeygn,"MSA, Not Principle City",1.0,2.0,0,1,0,0


Check the origional columns against the encoded ones to be clear which labels correspond to eachother

In [50]:
housing_df = h1n1_train[['rent_or_own', 'encoded_rent_or_own']]
housing_df

,rent_or_own,encoded_rent_or_own
respondent_id,,
22067,Own,0
2876,Rent,1
22939,Rent,1
4154,Rent,1
23434,Own,0
...,...,...
4682,Own,0
4285,Own,0
14877,Own,0


#### Own == 0
#### Rent == 1

In [51]:
married_df = h1n1_train[['marital_status', 'encoded_marital_status']]
married_df

,marital_status,encoded_marital_status
respondent_id,,
22067,Married,0
2876,Not Married,1
22939,Married,0
4154,Married,0
23434,Not Married,1
...,...,...
4682,Married,0
4285,Married,0
14877,Married,0


#### Married == 0
#### Not Married == 1

In [52]:
gender_df = h1n1_train[['sex', 'encoded_sex']]
gender_df

,sex,encoded_sex
respondent_id,,
22067,Male,1
2876,Female,0
22939,Female,0
4154,Male,1
23434,Male,1
...,...,...
4682,Male,1
4285,Female,0
14877,Female,0


#### Female == 0 
#### Male == 1

### One Hot Encode Remaining Columns

In [53]:
# Encode age_group column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(h1n1_train[['age_group']])

# nice columns for display
cols = ['age_group_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(h1n1_train[['age_group']]).todense()
h1n1_train = pd.concat([
    h1n1_train,
    pd.DataFrame(m, columns=cols, index=h1n1_train.index)
], axis=1)

m = encoder.transform(h1n1_test[['age_group']]).todense()
h1n1_test = pd.concat([
    h1n1_test,
    pd.DataFrame(m, columns=cols, index=h1n1_test.index)
], axis=1)

Check to make sure it worked

In [54]:
h1n1_train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,4.0,2.0,2.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0,1.0
2876,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,4.0,5.0,5.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.0,0.0,0,1,1,0,0.0,0.0,0.0,0.0,1.0
22939,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,2.0,1.0,0,1,0,0,0.0,0.0,1.0,0.0,0.0
4154,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,1.0,5.0,1.0,1.0,45 - 54 Years,< 12 Years,Other or Multiple,Male,"> $75,000",Married,Rent,Employed,mlyzmhmf,"MSA, Principle City",1.0,0.0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
23434,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,0,0,1,1,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,2.0,1.0,18 - 34 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",1.0,3.0,0,0,0,1,1.0,0.0,0.0,0.0,0.0
4285,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,4.0,1.0,45 - 54 Years,Some College,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Unemployed,bhuqouqj,"MSA, Not Principle City",3.0,0.0,0,0,0,0,0.0,0.0,1.0,0.0,0.0
14877,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,1.0,4.0,2.0,1.0,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",1.0,2.0,0,0,0,0,0.0,1.0,0.0,0.0,0.0


In [55]:
h1n1_test

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1372,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.0,0.0,0,0,1,0,0.0,0.0,0.0,1.0,0.0
21526,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,4.0,2.0,4.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Principle City",1.0,0.0,0,0,0,0,0.0,0.0,0.0,1.0,0.0
396,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,3.0,1.0,2.0,35 - 44 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,Non-MSA,1.0,2.0,0,0,0,1,0.0,1.0,0.0,0.0,0.0
11045,3.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,4.0,4.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,bhuqouqj,"MSA, Principle City",3.0,1.0,0,0,0,1,0.0,0.0,0.0,1.0,0.0
5072,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,2.0,2.0,2.0,65+ Years,College Graduate,White,Male,Below Poverty,Not Married,Rent,Not in Labor Force,oxchjgsf,"MSA, Principle City",0.0,0.0,0,1,1,1,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20153,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,4.0,4.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",3.0,0.0,0,0,0,0,0.0,0.0,0.0,1.0,0.0
8108,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,College Graduate,White,Female,"> $75,000",Not Married,Rent,Employed,dqpwygqj,"MSA, Principle City",1.0,0.0,1,1,1,0,0.0,0.0,0.0,1.0,0.0
19271,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0,5.0,5.0,5.0,55 - 64 Years,Some College,White,Female,Below Poverty,Married,Own,Employed,mlyzmhmf,Non-MSA,3.0,2.0,1,0,0,0,0.0,0.0,0.0,1.0,0.0


Excellent! It worked the way I wanted. Now I have a column with a 1 if the observation falls into that category and a zero if it does not. 

Now repeat for the remaining columns

In [56]:
# Encode education column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(h1n1_train[['education']])

# nice columns for display
cols = ['education_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(h1n1_train[['education']]).todense()
h1n1_train = pd.concat([
    h1n1_train,
    pd.DataFrame(m, columns=cols, index=h1n1_train.index)
], axis=1)

m = encoder.transform(h1n1_test[['education']]).todense()
h1n1_test = pd.concat([
    h1n1_test,
    pd.DataFrame(m, columns=cols, index=h1n1_test.index)
], axis=1)

In [57]:
# Encode race column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(h1n1_train[['race']])

# nice columns for display
cols = ['race_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(h1n1_train[['race']]).todense()
h1n1_train = pd.concat([
    h1n1_train,
    pd.DataFrame(m, columns=cols, index=h1n1_train.index)
], axis=1)

m = encoder.transform(h1n1_test[['race']]).todense()
h1n1_test = pd.concat([
    h1n1_test,
    pd.DataFrame(m, columns=cols, index=h1n1_test.index)
], axis=1)

In [58]:
# Encode income_poverty column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(h1n1_train[['income_poverty']])

# nice columns for display
cols = ['income_poverty_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(h1n1_train[['income_poverty']]).todense()
h1n1_train = pd.concat([
    h1n1_train,
    pd.DataFrame(m, columns=cols, index=h1n1_train.index)
], axis=1)

m = encoder.transform(h1n1_test[['income_poverty']]).todense()
h1n1_test = pd.concat([
    h1n1_test,
    pd.DataFrame(m, columns=cols, index=h1n1_test.index)
], axis=1)

In [59]:
h1n1_train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,2.0,4.0,2.0,2.0,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2876,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,4.0,5.0,5.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.0,0.0,0,1,1,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22939,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,2.0,1.0,0,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4154,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,1.0,5.0,1.0,1.0,45 - 54 Years,< 12 Years,Other or Multiple,Male,"> $75,000",Married,Rent,Employed,mlyzmhmf,"MSA, Principle City",1.0,0.0,0,1,0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
23434,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,2.0,1.0,18 - 34 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",1.0,3.0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4285,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,4.0,1.0,45 - 54 Years,Some College,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Unemployed,bhuqouqj,"MSA, Not Principle City",3.0,0.0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
14877,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,1.0,4.0,2.0,1.0,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",1.0,2.0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [60]:
h1n1_test

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1372,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.0,0.0,0,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
21526,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,4.0,2.0,4.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Principle City",1.0,0.0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
396,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,3.0,1.0,2.0,35 - 44 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,Non-MSA,1.0,2.0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
11045,3.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,4.0,4.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,bhuqouqj,"MSA, Principle City",3.0,1.0,0,0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5072,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,2.0,2.0,2.0,65+ Years,College Graduate,White,Male,Below Poverty,Not Married,Rent,Not in Labor Force,oxchjgsf,"MSA, Principle City",0.0,0.0,0,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20153,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,4.0,4.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",3.0,0.0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8108,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,5.0,2.0,1.0,55 - 64 Years,College Graduate,White,Female,"> $75,000",Not Married,Rent,Employed,dqpwygqj,"MSA, Principle City",1.0,0.0,1,1,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
19271,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0,5.0,5.0,5.0,55 - 64 Years,Some College,White,Female,Below Poverty,Married,Own,Employed,mlyzmhmf,Non-MSA,3.0,2.0,1,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


__Now perform the same encoding on the seasonal_train and test dfs__

In [61]:
# Encode age_group column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(seasonal_train[['age_group']])

# nice columns for display
cols = ['age_group_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(seasonal_train[['age_group']]).todense()
seasonal_train = pd.concat([
    seasonal_train,
    pd.DataFrame(m, columns=cols, index=seasonal_train.index)
], axis=1)

m = encoder.transform(seasonal_test[['age_group']]).todense()
seasonal_test = pd.concat([
    seasonal_test,
    pd.DataFrame(m, columns=cols, index=seasonal_test.index)
], axis=1)

In [62]:
# Encode education column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(seasonal_train[['education']])

# nice columns for display
cols = ['education_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(seasonal_train[['education']]).todense()
seasonal_train = pd.concat([
    seasonal_train,
    pd.DataFrame(m, columns=cols, index=seasonal_train.index)
], axis=1)

m = encoder.transform(seasonal_test[['education']]).todense()
seasonal_test = pd.concat([
    seasonal_test,
    pd.DataFrame(m, columns=cols, index=seasonal_test.index)
], axis=1)

In [63]:
# Encode race column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(seasonal_train[['race']])

# nice columns for display
cols = ['race_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(seasonal_train[['race']]).todense()
seasonal_train = pd.concat([
    seasonal_train,
    pd.DataFrame(m, columns=cols, index=seasonal_train.index)
], axis=1)

m = encoder.transform(seasonal_test[['race']]).todense()
seasonal_test = pd.concat([
    seasonal_test,
    pd.DataFrame(m, columns=cols, index=seasonal_test.index)
], axis=1)

In [64]:
# Encode income_poverty column

# Create encoder object
encoder = OneHotEncoder()

# Fit on the age_group column of the train df
encoder.fit(seasonal_train[['income_poverty']])

# nice columns for display
cols = ['income_poverty_' + c for c in encoder.categories_[0]]

# Transform the column on train and test and concatenate new df onto train and test dfs
m = encoder.transform(seasonal_train[['income_poverty']]).todense()
seasonal_train = pd.concat([
    seasonal_train,
    pd.DataFrame(m, columns=cols, index=seasonal_train.index)
], axis=1)

m = encoder.transform(seasonal_test[['income_poverty']]).todense()
seasonal_test = pd.concat([
    seasonal_test,
    pd.DataFrame(m, columns=cols, index=seasonal_test.index)
], axis=1)

Turn these into functions and add to prepare.py. I'll just do one example here to test if it works then comment it out to make sure I don't have duplicate columns moving forward. The other functions will only be added to the .py file.

In [65]:
# def ohe_income_poverty(train,test):
#     # Encode income_poverty column

#     # Create encoder object
#     encoder = OneHotEncoder()

#     # Fit on the age_group column of the train df
#     encoder.fit(train[['income_poverty']])

#     # nice columns for display
#     cols = ['income_poverty_' + c for c in encoder.categories_[0]]

#     # Transform the column on train and test and concatenate new df onto train and test dfs
#     m = encoder.transform(train[['income_poverty']]).todense()
#     train = pd.concat([
#         train,
#         pd.DataFrame(m, columns=cols, index=train.index)
#     ], axis=1)

#     m = encoder.transform(test[['income_poverty']]).todense()
#     test = pd.concat([
#         test,
#         pd.DataFrame(m, columns=cols, index=test.index)
#     ], axis=1)

#     return train, test

### Scaling

- The following columns are either opions on a scale of 1-5 or a count of number of children or adults in a household:
  - h1n1_concern
  - h1n1_knowledge
  - opinion_h1n1_vacc_effective
  - opinion_h1n1_risk
  - opinion_h1n1_sick_from_vacc
  - opinion_seas_vacc_effective
  - opinion_seas_risk
  - opinion_seas_sick_from_vac
  - household_adults
  - household_children
- All other features are on a scale of 0-1. I will apply a MinMax Scaler to the above columns to get them also on a 0-1 scale to avoid weighting issues in the models to come. 

In [66]:
# Create a scaler object using SKlearn's MinMax Scaler
scaler = MinMaxScaler()

In [67]:
# Add scaled columns to train dataframe
h1n1_train[['h1n1_concern','h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children'
      ]] = scaler.fit_transform(
    h1n1_train[['h1n1_concern',
       'h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children']])

Check to make sure scaling worked appropriately

In [69]:
h1n1_train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,0.333333,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.75,0.00,0.25,0.75,0.25,0.25,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.333333,0.000000,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2876,1.000000,0.5,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.00,0.75,1.00,1.00,0.75,0.25,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.000000,0.000000,0,1,1,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22939,0.666667,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.50,0.75,0.25,0.75,0.25,0.25,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,0.666667,0.333333,0,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4154,0.000000,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.75,0.00,0.00,1.00,0.00,0.00,45 - 54 Years,< 12 Years,Other or Multiple,Male,"> $75,000",Married,Rent,Employed,mlyzmhmf,"MSA, Principle City",0.333333,0.000000,0,1,0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
23434,0.000000,0.5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,0.25,0.25,0.75,0.75,0.25,65+ Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",0.333333,0.000000,0,0,1,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,0.333333,0.5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,0.25,0.25,0.75,0.25,0.00,18 - 34 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",0.333333,1.000000,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4285,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.50,0.50,0.00,0.00,0.75,0.00,45 - 54 Years,Some College,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Unemployed,bhuqouqj,"MSA, Not Principle City",1.000000,0.000000,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
14877,0.333333,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.00,0.25,0.00,0.75,0.25,0.00,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",0.333333,0.666667,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


Excellent! The MinMax Scaler was applied correctly. Now repeat this process for the test dataframe and turn these transformations into functions

In [79]:
# Add scaled columns to test dataframes
h1n1_test[['h1n1_concern','h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children'
      ]] = scaler.fit_transform(
    h1n1_test[['h1n1_concern',
       'h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children']])

In [73]:
# Add scaled columns to train dataframe
seasonal_train[['h1n1_concern','h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children'
      ]] = scaler.fit_transform(
   seasonal_train[['h1n1_concern',
       'h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children']])

In [74]:
# Add scaled columns to test dataframe
seasonal_test[['h1n1_concern','h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children'
      ]] = scaler.fit_transform(
   seasonal_test[['h1n1_concern',
       'h1n1_knowledge', 
       'opinion_h1n1_vacc_effective',
       'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc',
       'opinion_seas_vacc_effective',
       'opinion_seas_risk',
       'opinion_seas_sick_from_vacc',
       'household_adults',
       'household_children']])

Verify changes

In [76]:
h1n1_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22067,0.333333,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.75,0.00,0.25,0.75,0.25,0.25,65+ Years,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.333333,0.000000,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2876,1.000000,0.5,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.00,0.75,1.00,1.00,0.75,0.25,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Rent,Employed,bhuqouqj,"MSA, Principle City",0.000000,0.000000,0,1,1,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
22939,0.666667,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.50,0.75,0.25,0.75,0.25,0.25,45 - 54 Years,College Graduate,Other or Multiple,Female,"> $75,000",Married,Rent,Employed,mlyzmhmf,Non-MSA,0.666667,0.333333,0,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [77]:
h1n1_test.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,h1n1_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1372,0.666667,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.75,0.25,0.25,1.00,0.25,0.00,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",0.000000,0.000000,0,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
21526,0.666667,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.25,0.25,0.75,0.75,0.25,0.75,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Principle City",0.333333,0.000000,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
396,0.666667,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.50,0.00,0.25,0.50,0.00,0.25,35 - 44 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,Non-MSA,0.333333,0.666667,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [78]:
seasonal_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,seasonal_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22862,0.333333,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.25,0.00,45 - 54 Years,< 12 Years,Hispanic,Male,"<= $75,000, Above Poverty",Married,Rent,Not in Labor Force,kbazzjca,Non-MSA,0.000000,0.000000,0,1,0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
20471,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,0.75,0.25,1.00,0.75,0.00,18 - 34 Years,College Graduate,Other or Multiple,Male,"> $75,000",Not Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",0.666667,0.000000,0,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
11401,0.333333,0.5,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.00,1.00,1.00,0.75,0.75,0.75,18 - 34 Years,< 12 Years,Hispanic,Female,Below Poverty,Married,Rent,Not in Labor Force,atmpeygn,"MSA, Not Principle City",0.333333,0.666667,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [80]:
seasonal_train.head(3)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,seasonal_vaccine,encoded_rent_or_own,encoded_marital_status,encoded_sex,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,race_Black,race_Hispanic,race_Other or Multiple,race_White,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22862,0.333333,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.25,0.00,45 - 54 Years,< 12 Years,Hispanic,Male,"<= $75,000, Above Poverty",Married,Rent,Not in Labor Force,kbazzjca,Non-MSA,0.000000,0.000000,0,1,0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
20471,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,0.75,0.25,1.00,0.75,0.00,18 - 34 Years,College Graduate,Other or Multiple,Male,"> $75,000",Not Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",0.666667,0.000000,0,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
11401,0.333333,0.5,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.00,1.00,1.00,0.75,0.75,0.75,18 - 34 Years,< 12 Years,Hispanic,Female,Below Poverty,Married,Rent,Not in Labor Force,atmpeygn,"MSA, Not Principle City",0.333333,0.666667,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
